## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [12]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [13]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [14]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [15]:
params.kpi = 1


### Profit Mazimization - Begin 

In [16]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

20-07-23 11:51:11-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
20-07-23 11:51:11-INFO-creating combinations
20-07-23 11:51:11-INFO-2450	 nR*(nR-1)
20-07-23 11:51:11-INFO-Reduction of feasible pairs by 99.14%
20-07-23 11:51:11-INFO-Degree 2 	Completed
20-07-23 11:51:11-INFO-trips to extend at degree 2 : 55
20-07-23 11:51:11-INFO-At degree 2 feasible extensions found out of 7 searched
20-07-23 11:51:11-INFO-Degree 3 	Completed
20-07-23 11:51:11-INFO-trips to extend at degree 3 : 7
20-07-23 11:51:11-INFO-At degree 3 feasible extensions found out of 1 searched
20-07-23 11:51:11-INFO-Degree 4 	Completed
20-07-23 11:51:11-INFO-Max degree reached 4
20-07-23 11:51:11-INFO-Trips still possible to extend at degree 4 : 1


In [17]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

20-07-23 11:51:12-INFO-Matching 50 trips to 113 rides in order to min u_veh
20-07-23 11:51:12-INFO-Problem solution: Optimal. 
Total costs for single trips:         12,452 
reduced by matching to:               11,335


In [18]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],1.567500,84,1,[1.5675000000000001],"[0, 84]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44810823, 44814377]","[None, 0, 0]",node time req_id od 0 NaN...,84,0.84,1.273500,0.254700,1.0188
1,[1],2.702500,146,1,[2.7025],"[127, 146]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44848435, 1413910876]","[None, 1, 1]",node time req_id od 0 ...,146,1.46,2.191500,0.438300,1.7532
2,[2],3.905000,211,1,[3.9050000000000002],"[156, 211]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1569975267, 44825833]","[None, 2, 2]",node time req_id od 0 ...,211,2.11,3.166500,0.633300,2.5332
3,[3],1.268500,68,1,[1.2685],"[194, 68]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 2175715314, 44838797]","[None, 3, 3]",node time req_id od 0 ...,68,0.68,1.030500,0.206100,0.8244
4,[4],0.764500,41,1,[0.7645],"[241, 41]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44764023, 44775753]","[None, 4, 4]",node time req_id od 0 NaN...,41,0.41,0.621000,0.124200,0.4968
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,"[11, 13, 15]",14.862675,769,30,"[5.6609, 3.5146, 5.687175]","[715.0, 178, 183, 59, 6, 343]","[11, 13, 15]","[13, 11, 15]",3,108,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44880082, 1371031363, 4617340492, 44812...","[None, 11, 13, 15, 13, 11, 15]",node time req_id od 0 ...,769,7.69,9.716625,1.943325,7.7733
109,"[11, 9, 13]",13.112400,400,30,"[5.763275, 3.77765, 3.5714750000000004]","[669.5, 67, 110, 107, 110, 6]","[11, 9, 13]","[9, 11, 13]",3,109,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44880082, 1851699236, 1371031363, 44845...","[None, 11, 9, 13, 9, 11, 13]",node time req_id od 0 ...,400,4.00,8.355375,1.671075,6.6843
110,"[10, 13, 15]",15.492525,734,30,"[6.15425, 3.535075, 5.8032]","[722.5, 147, 183, 59, 105, 240]","[10, 13, 15]","[13, 10, 15]",3,110,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44856869, 1371031363, 4617340492, 44812...","[None, 10, 13, 15, 13, 10, 15]",node time req_id od 0 ...,734,7.34,9.848250,1.969650,7.8786
111,"[10, 9, 13]",13.519300,461,32,"[6.088275, 3.816325, 3.6147]","[677.0, 36, 110, 107, 103, 105]","[10, 9, 13]","[9, 13, 10]",3,111,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44856869, 1851699236, 1371031363, 44845...","[None, 10, 9, 13, 9, 13, 10]",node time req_id od 0 ...,461,4.61,8.487000,1.697400,6.7896


In [19]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

20-07-23 11:51:15-INFO-Matching 50 trips to 113 rides in order to min u_veh
20-07-23 11:51:15-INFO-Problem solution: Optimal. 
Total costs for single trips:         12,452 
reduced by matching to:               11,335
0.0005
20-07-23 11:51:18-CRITICAL-this is request 0 with [0] available rides.
20-07-23 11:51:18-CRITICAL-ride 0 available [1]
20-07-23 11:51:18-CRITICAL-this is reuqest 0 with [0] still available rides.
20-07-23 11:51:18-CRITICAL-vehicle 1 has 1 choices
0.0005
20-07-23 11:51:18-CRITICAL-this is request 1 with [1] available rides.
20-07-23 11:51:18-CRITICAL-ride 1 available [1]
20-07-23 11:51:18-CRITICAL-this is reuqest 1 with [1] still available rides.
20-07-23 11:51:18-CRITICAL-vehicle 13 has 1 choices
0.0005
20-07-23 11:51:18-CRITICAL-this is request 2 with [2] available rides.
20-07-23 11:51:18-CRITICAL-ride 2 available [1]
20-07-23 11:51:18-CRITICAL-this is reuqest 2 with [2] still available rides.
20-07-23 11:51:18-CRITICAL-vehicle 12 has 1 choices
0.0005
20-07-23 11

In [20]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,STARTS_REPOSITIONING,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM
1,4.0,0.0,False,0,0,2173.0,60.0,0,80.0,0,...,0,0,0,7312.0,556.0,0,498.0,260.0,5998.0,5.578
2,0.0,0.0,False,0,0,778.0,0.0,0,0.0,0,...,0,0,0,10021.0,0.0,0,0.0,0.0,10021.0,0.000
3,3.0,0.0,False,0,0,2011.0,30.0,0,46.0,0,...,0,0,0,7848.0,602.0,0,172.0,166.0,6908.0,9.213
4,2.0,0.0,False,0,0,2594.0,30.0,0,40.0,0,...,0,0,0,7521.0,461.0,0,93.0,130.0,6837.0,4.627
5,3.0,0.0,False,0,0,1796.0,30.0,0,109.0,0,...,0,0,0,7349.0,830.0,0,595.0,229.0,5695.0,12.275
6,3.0,0.0,False,0,0,2363.0,45.0,0,58.0,0,...,0,0,0,7795.0,331.0,0,117.0,193.0,7154.0,3.322
7,4.0,0.0,False,0,0,1592.0,22.0,0,74.0,0,...,0,0,0,7518.0,701.0,0,772.0,216.0,5829.0,12.865
8,4.0,0.0,False,0,0,1178.0,30.0,0,146.0,0,...,0,0,0,8652.0,362.0,0,311.0,296.0,7683.0,8.621
9,5.0,0.0,False,0,0,1325.0,37.0,0,103.0,0,...,0,0,0,7650.0,913.0,0,621.0,290.0,5826.0,13.462
10,2.0,0.0,False,0,0,2523.0,30.0,0,31.0,0,...,0,0,0,7719.0,349.0,0,87.0,121.0,7162.0,3.502


In [21]:
sim.vehs[1].rdf

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,sim_schedule,ttrav,dist,fare,commission,driver_revenue,pickup_dist,trav_dist,operating_cost,profit
0,[0],1.5675,84,1,[1.5675000000000001],"[0, 84]",[0],[0],1,0,...,node time req_id od 0 NaN...,84,0.84,1.2735,0.2547,1.0188,2071,2071.84,1.035920,-0.017120
16,[16],2.6840,145,1,[2.684],"[1075, 145]",[16],[16],1,16,...,node time req_id od 0 ...,145,1.45,2.1765,0.4353,1.7412,918,919.45,0.459725,1.281475
29,[29],3.8400,207,1,[3.84],"[2066, 207]",[29],[29],1,29,...,node time req_id od 0 ...,207,2.07,3.1155,0.6231,2.4924,1472,1474.07,0.737035,1.755365
47,[47],2.2215,120,1,[2.2215000000000003],"[3185, 120]",[47],[47],1,47,...,node time req_id od 0 ...,120,1.20,1.8015,0.3603,1.4412,1359,1360.20,0.680100,0.761100


In [22]:
pd.DataFrame(sim.vehs[1].myrides)

,veh,pos,t,event,paxes
0,1,1.371031e+09,0,STARTS_DAY,[]
1,1,1.371031e+09,0,OPENS_APP,[]
2,1,1.371031e+09,0,RECEIVES_REQUEST,[]
3,1,1.371031e+09,15,ACCEPTS_REQUEST,[]
4,1,1.371031e+09,35,IS_ACCEPTED_BY_TRAVELLER,[None]
5,1,4.481082e+07,223,ARRIVES_AT_PICKUP,[]
6,1,4.481082e+07,223,MEETS_TRAVELLER_AT_PICKUP,[]
7,1,4.481082e+07,253,DEPARTS_FROM_PICKUP,[0]
8,1,4.481438e+07,337,ARRIVES_AT_DROPOFF,[0]
9,1,4.481438e+07,1075,RECEIVES_REQUEST,[]


In [26]:
responses = []
avg_kpi = []
idle_time = []

for i in range(2, 3):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

0.0005
20-07-23 11:49:01-CRITICAL-this is request 0 with [0] available rides.
20-07-23 11:49:01-CRITICAL-ride 0 available [1]
20-07-23 11:49:01-CRITICAL-this is request 0 with [0] still available rides.
0.0005
20-07-23 11:49:01-CRITICAL-this is request 1 with [1] available rides.
20-07-23 11:49:01-CRITICAL-ride 1 available [1]
20-07-23 11:49:01-CRITICAL-this is request 1 with [1] still available rides.
0.0005
20-07-23 11:49:02-CRITICAL-this is request 2 with [2, 76, 77, 78, 79, 80] available rides.
20-07-23 11:49:02-CRITICAL-ride 2 available [1]
20-07-23 11:49:02-CRITICAL-ride 76 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 77 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 78 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 79 available [1, 0]
20-07-23 11:49:02-CRITICAL-ride 80 available [1, 0]
20-07-23 11:49:02-CRITICAL-this is request 2 with [2, 76, 77, 78, 79, 80] still available rides.
Pooled rides
20-07-23 11:49:02-CRITICAL-vehicle 6 has 6 choices
0.0005
20-07-23 11:49:02-CR